このプログラムは，研究データを引き継ぐためのものです
提案法の推定結果を印南さんのデータと同様のファイル形式で保存します．

In [1]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")
# Pkg.add("DataStructures")

In [2]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
using DataStructures
# using Base
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/MMR-W.jl");

In [3]:
# 評価基準数
num_criteria = [4, 5, 6, 7, 8];
# 真の区間重要度のセッティング
true_importance = ["A", "B", "C", "D", "E"]; # ["A", "B", "C", "D", "E"]

In [4]:
# 真の区間重要度を読み込み
for N in num_criteria
    for setting in true_importance

        # DataFrameを作成
        Simp_columns = OrderedDict()
        push!(Simp_columns, "Num" => Int[])
        for i in 1:N
            push!(Simp_columns, "wL[$(i-1)]" => Float64[])
            push!(Simp_columns, "wR[$(i-1)]" => Float64[])
        end
        push!(Simp_columns, "Sum_of_Width" => Float64[])
        Simp = DataFrame(Simp_columns)

        F_columns = OrderedDict()
        push!(F_columns, "Num" => Int[])
        for i in 1:N
            push!(F_columns, "F_$i" => Float64[])
        end
        push!(F_columns, "Average" => Float64[])
        index_Fi = DataFrame(F_columns)

        P_columns = OrderedDict()
        push!(P_columns, "Num" => Int[])
        for i in 1:N
            push!(P_columns, "P_$i" => Float64[])
        end
        push!(P_columns, "Average" => Float64[])
        index_Pi = DataFrame(P_columns)

        Q_columns = OrderedDict()
        push!(Q_columns, "Num" => Int[])
        for i in 1:N
            push!(Q_columns, "Q_$i" => Float64[])
        end
        push!(Q_columns, "Average" => Float64[])
        index_Qi = DataFrame(Q_columns)

        R_columns = OrderedDict()
        push!(R_columns, "Num" => Int[])
        for i in 1:N
            push!(R_columns, "R_$i" => Float64[])
        end
        push!(R_columns, "Average" => Float64[])
        index_Ri = DataFrame(R_columns)

        T_E_matome_all_columns = OrderedDict()
        push!(T_E_matome_all_columns, "Index" => String[])
        push!(T_E_matome_all_columns, "Mean" => Float64[])
        push!(T_E_matome_all_columns, "Var" => Float64[])
        push!(T_E_matome_all_columns, "SD" => Float64[])
        T_E_matome_all = DataFrame(T_E_matome_all_columns)

    
        df = CSV.File("./data/N"*string(N)*"_"*setting*"_Given_interval_weight.csv", header=false) |> DataFrame
        T = Vector{Interval{Float64}}()
        for i in 1:N
            push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
        end

        # 一対比較行列を1000個読み込み
        df1 = CSV.File("./data/N"*string(N)*"_a3_"*setting*"_PCM_int.csv", header=false) |> DataFrame

        # データフレームをN×Nのサブデータフレームに分割
        subdfs = split_dataframe(df1, N);
        # 結果格納用
        P = Matrix{Float64}(undef, N, 1000)
        Q = Matrix{Float64}(undef, N, 1000)
        R = Matrix{Float64}(undef, N, 1000)
        F = Matrix{Float64}(undef, N, 1000)

        # 1000個のデータで推定
        for (i, subdf) in enumerate(subdfs)

            # 一対比較行列を取り出す
            A = Matrix(subdf)

            # 提案手法で区間重要度推定
            solution = MMR_W(A, EV)
            E = solution.W # 区間重要度の推定値
            E_data = Vector{Float64}()
            for j in 1:N
                push!(E_data, inf(E[j]))
                push!(E_data, sup(E[j]))
            end
            push!(Simp, (i, E_data..., sum(c_diam.(E))))
            # display(L"E = %$(intervalVectorLaTeXString(E))")
            # display(L"T = %$(intervalVectorLaTeXString(T))")

            TcapE = T .∩ E # 真の区間重要度と推定値の積集合
            TcupE = T .∪ E # 真の区間重要度と推定値の和集合
            # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
            # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
            
            # P, Q, R, F の計算
            P[:, i] = calculate_P(T, E)
            Q[:, i] = calculate_Q(T, E)
            R[:, i] = calculate_R(T, E)
            F[:, i] = calculate_F(T, E)

            Pi_data = [P[j, i] for j in 1:N]
            push!(index_Pi, (i, Pi_data..., mean(P[:,i])))

            Qi_data = [Q[j, i] for j in 1:N]
            push!(index_Qi, (i, Qi_data..., mean(Q[:,i])))

            Ri_data = [R[j, i] for j in 1:N]
            push!(index_Ri, (i, Ri_data..., mean(R[:,i])))

            Fi_data = [F[j, i] for j in 1:N]
            push!(index_Fi, (i, Fi_data..., mean(F[:,i])))

        end

        println("N = ", N)
        println("setting = ", setting)

        P_ave = mean(P)
        Q_ave = mean(Q)
        R_ave = mean(R)
        F_ave = mean(F)

        P_var = var(P)
        Q_var = var(Q)
        R_var = var(R)
        F_var = var(F)

        # DataFrameにデータを追加
        push!(T_E_matome_all, ("P", P_ave, P_var, sqrt(P_var)))
        push!(T_E_matome_all, ("Q", Q_ave, Q_var, sqrt(Q_var)))
        push!(T_E_matome_all, ("R", R_ave, R_var, sqrt(R_var)))
        push!(T_E_matome_all, ("F", F_ave, F_var, sqrt(F_var)))

        root_folder = "./out/Simp/N=" * string(N) * "/a3/" * setting
        output_path = root_folder * "/E-MMRw"
        # ディレクトリが存在しない場合にのみディレクトリを作成
        if !isdir(output_path)
            mkdir(output_path)
        end

        Simp_output_path = output_path * "/Simp.csv"
        index_Fi_output_path = output_path * "/index_Fi.csv"
        index_Pi_output_path = output_path * "/index_Pi.csv"
        index_Qi_output_path = output_path * "/index_Qi.csv"
        index_Ri_output_path = output_path * "/index_Ri.csv"
        T_E_matome_all_output_path = output_path * "/T_E_matome_all.csv"

        CSV.write(Simp_output_path, Simp)
        CSV.write(index_Fi_output_path, index_Fi)
        CSV.write(index_Pi_output_path, index_Pi)
        CSV.write(index_Qi_output_path, index_Qi)
        CSV.write(index_Ri_output_path, index_Ri)
        CSV.write(T_E_matome_all_output_path, T_E_matome_all)

    end
end



N = 4
setting = A


N = 4
setting = B


N = 4
setting = C


N = 4
setting = D


N = 4
setting = E


...

N = 8
setting = A


N = 8
setting = B


N = 8
setting = C


N = 8
setting = D


N = 8
setting = E
